In [1]:
import sys
import os
from datetime import datetime
import json
import shutil
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm_notebook, tqdm
from matplotlib import pyplot as plt

sys.path.append('../src/')
from env import BTCMarket_Env
from agent import Trader_Agent
from DQNtrainer import DQNTrainer
from DRLtrainer import DRLTrainer
from collections import deque
import h5py
from itertools import product


2023-03-28 15:07:41.621071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 15:07:41.773729: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/arnold/systemc-2.3.3/lib-linux64
2023-03-28 15:07:41.773755: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-28 15:07:41.802775: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-28

In [2]:
hpo_params= { 
    'obs_space' : [(8,20)], 
    'action_domain' : [(0.0,1.0)], # (-1.0,1.0),
    'money' : [10000], 
    'fee' : [0.001], 
    'asset' : ['BTC', 'PERP'],
#    'reward_function' : ['compute_reward_from_tutor', 'reward_sharpe_ratio', 'reward_sortino_ratio', 
#                        'reward_differential_sharpe_ratio', 'reward_sterling_ratio'],
    'reward_function' : ['compute_reward_from_tutor'],
    
    'learning_rate': [1e-3],
    'trainer' : ['DRLTrainer'], # 'DRLTrainer', 'DQNTrainer'
    'episodes' : [1], 
    'runs_p_eps' : [1], 
    'batch_size': [1],
    'epoch': [5],
    'gamma': [0.95],
    
    'epsilon': [0.7],
    'epsilon_final':[0.01],
    'epsilon_decay':[0.995],
}


keys, values = zip(*hpo_params.items())
hpo_list = [dict(zip(keys, v)) for v in product(*values)]
print(hpo_list)

[{'obs_space': (8, 20), 'action_domain': (0.0, 1.0), 'money': 10000, 'fee': 0.001, 'asset': 'BTC', 'reward_function': 'compute_reward_from_tutor', 'learning_rate': 0.001, 'trainer': 'DRLTrainer', 'episodes': 1, 'runs_p_eps': 1, 'batch_size': 1, 'epoch': 5, 'gamma': 0.95, 'epsilon': 0.7, 'epsilon_final': 0.01, 'epsilon_decay': 0.995}, {'obs_space': (8, 20), 'action_domain': (0.0, 1.0), 'money': 10000, 'fee': 0.001, 'asset': 'PERP', 'reward_function': 'compute_reward_from_tutor', 'learning_rate': 0.001, 'trainer': 'DRLTrainer', 'episodes': 1, 'runs_p_eps': 1, 'batch_size': 1, 'epoch': 5, 'gamma': 0.95, 'epsilon': 0.7, 'epsilon_final': 0.01, 'epsilon_decay': 0.995}]


In [3]:
i = 0
#for i, param_combination in enumerate(hpo_list):
param_combination = hpo_list[0]
if param_combination.get('trainer') == 'DQNTrainer':
    action_space = 4
    algorithm = f'DQN_trial_{i}'
else:
    action_space = 1
    algorithm = f'DRL_trial_{i}'
    
param_combination['action_space'] = action_space
param_combination['algorithm'] = algorithm
print(param_combination)

env = BTCMarket_Env(observation_space = param_combination.get('obs_space', (8,20)),
        action_space = action_space,
        start_money = param_combination.get('money', 10000),
        trading_fee = param_combination.get('fee', (8,20)),
        asset = param_combination.get('asset', 'PERP'),
        reward_function = param_combination.get('reward_function', 'reward_differential_sharpe_ratio'),
                   )

agent = Trader_Agent(observation_space = param_combination.get('obs_space', (8,20)),
            action_space = action_space,
            epsilon = param_combination.get('epsilon', 0.7),
            epsilon_final = param_combination.get('epsilon_final', 0.01),
            epsilon_decay = param_combination.get('epsilon_decay', 0.995),
                    )

trainer_class = eval(param_combination.get('trainer', 'DRLTrainer'))
trainer = trainer_class(env, agent,
            observation_space = param_combination.get('obs_space', (8,20)),
            action_space = action_space,
            batch_size=param_combination.get('batch_size', 50),
            epoch=param_combination.get('epoch', 5),
            gamma=param_combination.get('gamma', 0.95),
            learning_rate=param_combination.get('learning_rate', 1e-3),
            algorithm=algorithm,
            lstm_path="./../notebooks/best_models/11_mar_2023/best_model_sequential_20back_10ahead.h5",
            # best_model_sequential_20back_10ahead lstm_2,
                       )

os.makedirs(trainer.train_folder,exist_ok=True)
with open(f'{trainer.train_folder}/params.json', 'w') as fp:
    json.dump(param_combination, fp)
    
trainer.rollout(n_episodes=param_combination.get('episodes', 2), 
                   run_per_episode=param_combination.get('runs_p_eps', 2))

{'obs_space': (8, 20), 'action_domain': (0.0, 1.0), 'money': 10000, 'fee': 0.001, 'asset': 'BTC', 'reward_function': 'compute_reward_from_tutor', 'learning_rate': 0.001, 'trainer': 'DRLTrainer', 'episodes': 1, 'runs_p_eps': 1, 'batch_size': 1, 'epoch': 5, 'gamma': 0.95, 'epsilon': 0.7, 'epsilon_final': 0.01, 'epsilon_decay': 0.995, 'action_space': 1, 'algorithm': 'DRL_trial_0'}


2023-03-28 15:07:47.380115: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/arnold/systemc-2.3.3/lib-linux64
2023-03-28 15:07:47.380178: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-28 15:07:47.380202: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-0OFSB15): /proc/driver/nvidia/version does not exist
2023-03-28 15:07:47.380488: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 8)]      0           []                               
                                                                                                  
 lstm_inputs (Lambda)           (None, 20, 4)        0           ['input_1[0][0]']                
                                                                                                  
 sequential (Sequential)        (None, 10)           1953940     ['lstm_inputs[0][0]']            
                                                                                                  
 flatten_1 (Flatten)            (None, 10)           0           ['sequential[0][0]']             
                                                                                              

  0%|                                                                                 | 0/671 [00:00<?, ?it/s]

Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: -0.9983
Epoch 2/5
1/1 [==============================] - 0s 27ms/step - loss: -0.9983
Epoch 3/5
1/1 [==============================] - 0s 36ms/step - loss: -0.9983
Epoch 4/5
1/1 [==============================] - 0s 38ms/step - loss: -0.9983
Epoch 5/5
1/1 [==============================] - 0s 33ms/step - loss: -0.9983


...

Epoch 1/5
1/1 [==============================] - 0s 22ms/step - loss: -0.9564
Epoch 2/5
1/1 [==============================] - 0s 30ms/step - loss: -0.9564
Epoch 3/5
1/1 [==============================] - 0s 28ms/step - loss: -0.9564
Epoch 4/5
1/1 [==============================] - 0s 30ms/step - loss: -0.9564
Epoch 5/5
1/1 [==============================] - 0s 29ms/step - loss: -0.9564


 22%|███████████████▊                                                       | 150/671 [00:30<01:15,  6.90it/s]

Epoch 1/5
1/1 [==============================] - 0s 25ms/step - loss: -0.9507
Epoch 2/5
1/1 [==============================] - 0s 25ms/step - loss: -0.9507
Epoch 3/5
1/1 [==============================] - 0s 28ms/step - loss: -0.9507
Epoch 4/5
1/1 [==============================] - 0s 28ms/step - loss: -0.9507
Epoch 5/5
1/1 [==============================] - 0s 27ms/step - loss: -0.9507


 23%|████████████████                                                       | 152/671 [00:30<01:43,  4.99it/s]


KeyboardInterrupt: 